# Implementing the data generators with TensorFlow
* Read data before preprocessing
* Build a tokenizer
* Refined tokenizer
* Generating skip-grams from the corpus
* Generating negative candidates
* Generating positive and negative candidates
* Frequencies to compute sampling table

In [34]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
%matplotlib inline
import zipfile
import re
import numpy as np
import pandas as pd
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='3'
import random
import tensorflow as tf
import matplotlib.pyplot as plt
from six.moves.urllib.request import urlretrieve
from sklearn.manifold import TSNE
from adjustText import adjust_text

# Downloading the data

In [35]:
url = 'http://mlg.ucd.ie/files/datasets/bbc-fulltext.zip'


def download_data(url, data_dir):
    """Download a file if not present, and make sure it's the right size."""

    os.makedirs(data_dir, exist_ok=True)

    file_path = os.path.join(data_dir, 'bbc-fulltext.zip')

    if not os.path.exists(file_path):
        print('Downloading file...')
        filename, _ = urlretrieve(url, file_path)
    else:
        print("File already exists")

    extract_path = os.path.join(data_dir, 'bbc')
    if not os.path.exists(extract_path):

        with zipfile.ZipFile(os.path.join(data_dir, 'bbc-fulltext.zip'), 'r') as zipf:
            zipf.extractall(data_dir)

    else:
        print("bbc-fulltext.zip has already been extracted")

download_data(url, 'data')

File already exists
bbc-fulltext.zip has already been extracted


# Reading data without preprocessing

In [36]:
def read_data(data_dir):

    # This will contain the full list of stories
    news_stories = []

    print("Reading files")

    i = 0 # Just used for printing progress
    for root, dirs, files in os.walk(data_dir):

        for fi, f in enumerate(files):

            # We don't read the README file
            if 'README' in f:
                continue

            # Printing progress
            i += 1
            print("."*i, f, end='\r')

            # Open the file
            with open(os.path.join(root, f), encoding='latin-1') as f:

                story = []
                # Read all the lines
                for row in f:

                    story.append(row.strip())

                # Create a single string with all the rows in the doc
                story = ' '.join(story)
                # Add that to the list
                news_stories.append(story)

        print('', end='\r')

    print(f"\nDetected {len(news_stories)} stories")
    return news_stories


news_stories = read_data(os.path.join('data', 'bbc'))

# Printing some stats and sample data
print(f"{sum([len(story.split(' ')) for story in news_stories])} words found in the total news set")
print('Example words (start): ',news_stories[0][:50])
print('Example words (end): ',news_stories[-1][-50:])


Reading files
..........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

# Building a tokenizer

In [37]:
from tensorflow.keras.preprocessing.text import Tokenizer
# from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(
    num_words=None,
    filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
    lower=True, split=' '
)

tokenizer.fit_on_texts(news_stories)
# tokenizer.fit_on_texts(news_stories)
print("Data fitted on the tokenizer")

Data fitted on the tokenizer


# Exploring the tokenizer

In [38]:
n_vocab = len(tokenizer.word_index.items()) + 1
print(f"Vocabulary size: {n_vocab}")

print("\nWords at the top")
print('\t', dict(list(tokenizer.word_index.items())[:10]))
print("\nWords at the bottom")
print('\t', dict(list(tokenizer.word_index.items())[-10:]))

Vocabulary size: 32360

Words at the top
	 {'the': 1, 'to': 2, 'of': 3, 'and': 4, 'a': 5, 'in': 6, 'for': 7, 'is': 8, 'that': 9, 'on': 10}

Words at the bottom
	 {"irishman's": 32350, 'sidefooting': 32351, '110th': 32352, 'pellegrino': 32353, 'pidgeley': 32354, "llanelli's": 32355, 'pectoral': 32356, 'signings': 32357, 'departures': 32358, 'invigorate': 32359}


# Building a tokenizer (refined)

In [39]:
from tensorflow.keras.preprocessing.text import Tokenizer

n_vocab = 15000 + 1
tokenizer = Tokenizer(
    num_words=n_vocab-1,
    filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
    lower=True, split=' ', oov_token='',
)

tokenizer.fit_on_texts(news_stories)
print("Data fitted on the tokenizer")

Data fitted on the tokenizer


# Checking the results of the tokenizer

In [40]:
print(f"Original: {news_stories[0][:100]}")
print(f"Sequence IDs: {tokenizer.texts_to_sequences([news_stories[0][:100]])[0]}")

Original: Snow Patrol feted at Irish awards  Snow Patrol were the big winners in Ireland's top music honours, 
Sequence IDs: [3596, 5574, 12988, 21, 760, 306, 3596, 5574, 43, 2, 229, 1143, 7, 2391, 146, 100, 2670]


# Converting all articles to word ID sequences

In [41]:
news_sequences = tokenizer.texts_to_sequences(news_stories)

# Generating skip-grams from the corpus

In [42]:
sample_word_ids = news_sequences[0][:5]
sample_phrase = ' '.join([tokenizer.index_word[wid] for wid in sample_word_ids])
print(f"Sample phrase: {sample_phrase}")
print(f"Sample word IDs: {sample_word_ids}\n")

window_size = 1 # How many words to consider left and right.

inputs, labels = tf.keras.preprocessing.sequence.skipgrams(
    sample_word_ids,
    vocabulary_size=n_vocab,
    window_size=window_size, negative_samples=1.0, shuffle=False,
    categorical=False, sampling_table=None, seed=None
)

print("Sample skip-grams")

for inp, lbl in zip(inputs, labels):
    print(f"\tInput: {inp} ({[tokenizer.index_word[wi] for wi in inp]}) / Label: {lbl}")

Sample phrase: snow patrol feted at irish
Sample word IDs: [3596, 5574, 12988, 21, 760]

Sample skip-grams
	Input: [3596, 5574] (['snow', 'patrol']) / Label: 1
	Input: [5574, 3596] (['patrol', 'snow']) / Label: 1
	Input: [5574, 12988] (['patrol', 'feted']) / Label: 1
	Input: [12988, 5574] (['feted', 'patrol']) / Label: 1
	Input: [12988, 21] (['feted', 'at']) / Label: 1
	Input: [21, 12988] (['at', 'feted']) / Label: 1
	Input: [21, 760] (['at', 'irish']) / Label: 1
	Input: [760, 21] (['irish', 'at']) / Label: 1
	Input: [12988, 2556] (['feted', 'listed']) / Label: 0
	Input: [760, 4793] (['irish', 'indoors']) / Label: 0
	Input: [5574, 12510] (['patrol', 'rescued']) / Label: 0
	Input: [12988, 3683] (['feted', 'puts']) / Label: 0
	Input: [21, 11218] (['at', 'farce']) / Label: 0
	Input: [3596, 7917] (['snow', 'corners']) / Label: 0
	Input: [5574, 6051] (['patrol', 'viewed']) / Label: 0
	Input: [21, 4744] (['at', 'tune']) / Label: 0


# Generating negative candidates

In [43]:
inputs, labels = tf.keras.preprocessing.sequence.skipgrams(
    sample_word_ids,
    vocabulary_size=len(tokenizer.word_index.items())+1,
    window_size=window_size, negative_samples=0, shuffle=False,
)

inputs, labels = np.array(inputs), np.array(labels)

negative_sampling_candidates, true_expected_count, sampled_expected_count = tf.random.log_uniform_candidate_sampler(
    # A true context word that appears in the context of the target
    true_classes=inputs[:1,1:], # [b, 1] sized tensor
    num_true=1, # number of true words per example
    num_sampled=10,
    unique=True,
    range_max=n_vocab,
    name="negative_sampling"
)

print(f"Positive sample: {inputs[:1,1:]}")
print(f"Negative samples: {negative_sampling_candidates}")
print(f"true_expected_count: {true_expected_count}")
print(f"sampled_expected_count: {sampled_expected_count}")

Positive sample: [[5574]]
Negative samples: [ 5165  1600   677   753  1212 11801  3537  5949    11     0]
true_expected_count: [[0.00018652]]
sampled_expected_count: [2.0128522e-04 6.4935384e-04 1.5327049e-03 1.3783171e-03 8.5697579e-04
 8.8111847e-05 2.9389292e-04 1.7476515e-04 8.3239615e-02 7.2083151e-01]


In [44]:
sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(n_vocab, sampling_factor=1e-05)

# Generating data (positive + negative candidates)

In [45]:
def skip_gram_data_generator(sequences, window_size, batch_size, negative_samples, vocab_size, seed=None):

    rand_sequence_ids = np.arange(len(sequences))
    np.random.shuffle(rand_sequence_ids)


    for si in rand_sequence_ids:

        positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
            sequences[si],
            vocabulary_size=vocab_size,
            window_size=window_size,
            negative_samples=0.0,
            shuffle=False,
            sampling_table=sampling_table,
            seed=seed
        )

        targets, contexts, labels = [], [], []

        for target_word, context_word in positive_skip_grams:
            context_class = tf.expand_dims(tf.constant([context_word], dtype="int64"), 1)

            negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
              true_classes=context_class,
              num_true=1,
              num_sampled=negative_samples,
              unique=True,
              range_max=vocab_size,
              name="negative_sampling")

            # Build context and label vectors (for one target word)
            context = tf.concat(
                [tf.constant([context_word], dtype='int64'), negative_sampling_candidates],
                axis=0
            )

            label = tf.constant([1] + [0]*negative_samples, dtype="int64")

            # Append each element from the training example to global lists.
            targets.extend([target_word]*(negative_samples+1))
            contexts.append(context)
            labels.append(label)

        contexts, targets, labels = np.concatenate(contexts), np.array(targets), np.concatenate(labels)

        assert contexts.shape[0] == targets.shape[0]
        assert contexts.shape[0] == labels.shape[0]

        # If seed is not provided, generate a random one
        if not seed:
            seed = random.randint(0, int(10e6))

        np.random.seed(seed)
        np.random.shuffle(contexts)
        np.random.seed(seed)
        np.random.shuffle(targets)
        np.random.seed(seed)
        np.random.shuffle(labels)


        for eg_id_start in range(0, contexts.shape[0], batch_size):
            yield (
                targets[eg_id_start: min(eg_id_start+batch_size, targets.shape[0])],
                contexts[eg_id_start: min(eg_id_start+batch_size, contexts.shape[0])]
            ), labels[eg_id_start: min(eg_id_start+batch_size, labels.shape[0])]


news_skip_gram_gen = skip_gram_data_generator(
    news_sequences, 4, 10, 5, n_vocab
)

for btc, bl in (news_skip_gram_gen):

    print(btc)
    print(bl)

    break

(array([ 1089,  1950,  3427,  4508,  1297,  2438, 11192,  4223,  3427,
       11192]), array([   0,  385,    2,   20,    5,  159, 4099,  147,   40,  634]))
[0 0 0 1 1 0 0 0 0 0]


# Defining hyperparameters

In [46]:
batch_size = 4096 # Data points in a single batch

embedding_size = 128 # Dimension of the embedding vector.

window_size=1 # We use a window size of n on either side of target word
negative_samples = 4 # Number of negative samples generated per example

epochs = 5 # Number of epochs to train for

# We pick a random validation set to sample nearest neighbors
valid_size = 16 # Random set of words to evaluate similarity on.
# We sample valid data points randomly from a large window without always being deterministic
valid_window = 250

# When selecting valid examples, we select some of the most frequent words as well as
# some moderately rare words
np.random.seed(54321)
random.seed(54321)

valid_term_ids = np.array(random.sample(range(valid_window), valid_size))
valid_term_ids = np.append(
    valid_term_ids, random.sample(range(1000, 1000+valid_window), valid_size),
    axis=0
)

# Defining the model

In [47]:
import tensorflow.keras.backend as K

K.clear_session()

# Inputs - skipgrams() function outputs target, context in that order
# we will use the same order
input_1 = tf.keras.layers.Input(shape=(), name='target')
input_2 = tf.keras.layers.Input(shape=(), name='context')

# Two embeddings layers are used, one for the context and one for the target
context_embedding_layer = tf.keras.layers.Embedding(
    input_dim=n_vocab, output_dim=embedding_size, name='context_embedding'
)
target_embedding_layer = tf.keras.layers.Embedding(
    input_dim=n_vocab, output_dim=embedding_size, name='target_embedding'
)

# Look up outputs of the embedding layers
target_out = target_embedding_layer(input_1)
context_out = context_embedding_layer(input_2)

# Computing the dot product between the two
out = tf.keras.layers.Dot(axes=-1)([context_out, target_out])

# Defining the model
skip_gram_model = tf.keras.models.Model(inputs=[input_1, input_2],
                                        outputs=out, name='skip_gram_model')

# Compiling the model
skip_gram_model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                        optimizer='adam', metrics=['accuracy'])

skip_gram_model.summary()

Model: "skip_gram_model"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ context             │ (None)            │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ target (InputLayer) │ (None)            │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ context_embedding   │ (None, 128)       │  1,920,128 │ context[0][0]     │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ target_embedding    │ (None, 128)       │  1,920,128 │ target[0][0]      │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dot (Dot)           │ (None, 1)         │          0 │ context_embeddin… │
│                     │                   │            │ target_embedding… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 3,840,256 (14.65 MB)

 Trainable params: 3,840,256 (14.65 MB)

 Non-trainable params: 0 (0.00 B)

# Training the model

In [48]:
class ValidationCallback(tf.keras.callbacks.Callback):

    def __init__(self, valid_term_ids, model_with_embeddings, tokenizer):

        self.valid_term_ids = valid_term_ids
        self.model_with_embeddings = model_with_embeddings
        self.tokenizer = tokenizer

        super().__init__()

    def on_epoch_end(self, epoch, logs=None):
        """ Validation logic """

        # We will use context embeddings to get the most similar words
        # Other strategies include: using target embeddings, mean embeddings after avaraging context/target
        embedding_weights = self.model_with_embeddings.get_layer("context_embedding").get_weights()[0]
        normalized_embeddings = embedding_weights / np.sqrt(np.sum(embedding_weights**2, axis=1, keepdims=True))

        # Get the embeddings corresponding to valid_term_ids
        valid_embeddings = normalized_embeddings[self.valid_term_ids, :]

        # Compute the similarity between valid_term_ids and all the embeddings
        # V x d (d x D) => V x D
        top_k = 5 # Top k items will be displayed
        similarity = np.dot(valid_embeddings, normalized_embeddings.T)

        # Invert similarity matrix to negative
        # Ignore the first one because that would be the same word as the probe word
        similarity_top_k = np.argsort(-similarity, axis=1)[:, 1: top_k+1]

        # Print the output
        for i, term_id in enumerate(valid_term_ids):

            similar_word_str = ', '.join([self.tokenizer.index_word[j] for j in similarity_top_k[i, :] if j >= 1])
            print(f"{self.tokenizer.index_word[term_id]}: {similar_word_str}")

        print('\n')


# Running the skip-gram algorithm

In [49]:
skipgram_validation_callback = ValidationCallback(valid_term_ids, skip_gram_model, tokenizer)

for ei in range(epochs):

    print(f"Epoch: {ei+1}/{epochs} started")

    news_skip_gram_gen = skip_gram_data_generator(
        news_sequences, window_size, batch_size, negative_samples, n_vocab
    )

    skip_gram_model.fit(
        news_skip_gram_gen, epochs=1,
        callbacks=skipgram_validation_callback,
    )

Epoch: 1/5 started
   2232/Unknown 97s 43ms/step - accuracy: 0.8001 - loss: 0.6282election: flat, afghanistan, end, success, order
me: him, users, things, do, authorities
with: between, against, global, since
you: we, they, don't, really, tories
were: renton, about, regions, if
win: likely, move, couple, made
those: trying, do, terrorists, find, sit
music: part, win, end, crucial
also: them, never, me, properties, help
international: attempting, shot, control, published, lay
best: part, leadership, allowed, powerful
him: users, things, tories, do, see
too: way, tories, going, thought, likely
some: trying, couple, comments, probably
through: going, difference, example, represent
mr: said, tony, gordon, michael, david
looked: consider, focus, those, trying, what
concern: due, measures, apologise, wealth, find
computers: apologise, convention, hold, try, part
along: designed, information, plans, powerful, handhelds
trust: focus, likely, example, embargo
row: convention, comment, health, m

/usr/local/python/3.12.1/lib/python3.12/contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


   2233/Unknown 98s 44ms/step - accuracy: 0.8057 - loss: 0.4587election: november's, workers, labour's, success, leadership
me: him, likhovtseva, tories, we'll, help
with: between, against, fenice, balco, global
you: we, they, don't, really, never
were: are, was, already, am, have
win: none, chance, napier, reaching, 61
those: able, trying, seriousness, listen, deals
music: asia, luxury, halt, uk's, total
also: already, never, widely, had, we've
international: juventus, customs, clash, today's, chorlton
best: eurovision, boogeyman, formula, halt, budget'
him: me, tories, them, help, things
too: bit, very, roddick's, adopting, fun
some: chocolate, draws, calibre, thrust, yourself
through: onto, rochdale, severn, dotted
mr: gordon, said, tony, robert, michael
looked: inspector, definitely, confirmed, staked, fury
concern: youngster, spying, paver, unity, idea
computers: le, precious, tonge, withdrawal, medicine
along: hearings, betrayal, mmorpgs, eventual, kodak
trust: approximately, sto